In [1]:
import os
from __future__ import print_function
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [2]:
num_classes=5
img_rows,img_col=48,48
batch_size=32

In [3]:
train= (r'C:\Users\infinity\Desktop\face-expression-recognition-dataset\face-expression-recognition-dataset\train')
validation= (r'C:\Users\infinity\Desktop\face-expression-recognition-dataset\face-expression-recognition-dataset\validation')

In [4]:
train_data_generator= ImageDataGenerator(rescale=1./255, rotation_range=30,shear_range=0.3, zoom_range=0.3, width_shift_range=0.4, 
                                         height_shift_range=0.4)
validation_data_generator=ImageDataGenerator(rescale=1./255)

In [5]:
train_generator=train_data_generator.flow_from_directory(train, color_mode='grayscale', target_size=(img_rows,img_col),
                                                         batch_size= batch_size, class_mode='categorical', shuffle=True)


validation_generator=validation_data_generator.flow_from_directory(validation,  color_mode='grayscale', target_size=(img_rows,img_col),
                                 batch_size= batch_size, class_mode='categorical', shuffle=True )

Found 24282 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [6]:
model= Sequential()

In [8]:
#Block 1
model.add(Conv2D(32,(3,3),padding='same', kernel_initializer='he_normal', input_shape=(img_rows,img_col,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same', kernel_initializer='he_normal', input_shape=(img_rows,img_col,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [9]:
#Block 2
model.add(Conv2D(64,(3,3),padding='same', kernel_initializer='he_normal', input_shape=(img_rows,img_col,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same', kernel_initializer='he_normal', input_shape=(img_rows,img_col,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [10]:
#Block 3
model.add(Conv2D(128,(3,3),padding='same', kernel_initializer='he_normal', input_shape=(img_rows,img_col,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same', kernel_initializer='he_normal', input_shape=(img_rows,img_col,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [11]:
#Block 4
model.add(Conv2D(256,(3,3),padding='same', kernel_initializer='he_normal', input_shape=(img_rows,img_col,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same', kernel_initializer='he_normal', input_shape=(img_rows,img_col,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [12]:
#Flattening Block
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [13]:
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [14]:

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                        

In [16]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [17]:
checkpoint= ModelCheckpoint('Emotion_detector_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
earlystop= EarlyStopping(monitor='val_loss', min_delta=0, patience=3, restore_best_weights=True, verbose=1 )
reduce_lr= ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_delta=0.0001 )

In [18]:
callbacks= [earlystop, checkpoint, reduce_lr]

In [19]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics= ['accuracy'])


C:\Users\infinity\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [22]:
nb_train_samples = 24176
nb_validation_samples = 3006
epochs=5

In [23]:
history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

<ipython-input-23-d7d1b8a23df7>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(


Epoch 1/5
755/755 [==============================] - ETA: 0s - loss: 1.8584 - accuracy: 0.2420
Epoch 00001: val_loss improved from inf to 1.55498, saving model to Emotion_detector_model.h5
755/755 [==============================] - 850s 1s/step - loss: 1.8584 - accuracy: 0.2420 - val_loss: 1.5550 - val_accuracy: 0.3125 - lr: 0.0010
Epoch 2/5
755/755 [==============================] - ETA: 0s - loss: 1.5770 - accuracy: 0.2788
Epoch 00002: val_loss improved from 1.55498 to 1.53380, saving model to Emotion_detector_model.h5
755/755 [==============================] - 335s 443ms/step - loss: 1.5770 - accuracy: 0.2788 - val_loss: 1.5338 - val_accuracy: 0.3078 - lr: 0.0010
Epoch 3/5
755/755 [==============================] - ETA: 0s - loss: 1.5477 - accuracy: 0.3015
Epoch 00003: val_loss improved from 1.53380 to 1.47344, saving model to Emotion_detector_model.h5
755/755 [==============================] - 327s 433ms/step - loss: 1.5477 - accuracy: 0.3015 - val_loss: 1.4734 - val_accuracy: 0.35